In [25]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

2025-05-04:18:16:42,339 INFO     [workspace.py:291] Found the config file in: /config.json


Workspace name: quick-starts-ws-279736
Azure region: eastus2
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-279736


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    print("Creating new cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_nodes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Creating new cluster
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
pip install 'azureml-sdk[notebooks]'

Note: you may need to restart the kernel to use updated packages.


In [10]:
#from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                script='train_wv.py',
                compute_target='Udacity',
                environment=sklearn_env)

#primary_metric = 'Accuracy'
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=100,
                max_concurrent_runs=4)

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
#RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_efddd267-7aa5-41a4-ab06-d137ed425fa4
Web View: https://ml.azure.com/runs/HD_efddd267-7aa5-41a4-ab06-d137ed425fa4?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-279736/workspaces/quick-starts-ws-279736&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-05-04T17:03:11.9259334Z][GENERATOR][WARNING]Space size : 12 is less than max total jobs : 100, only 12 jobs will be generated 
[2025-05-04T17:03:13.6083620Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-05-04T17:03:14.2235743Z][SCHEDULER][INFO]Scheduling job, id='HD_efddd267-7aa5-41a4-ab06-d137ed425fa4_0' 
[2025-05-04T17:03:14.2227403Z][SCHEDULER][INFO]Scheduling job, id='HD_efddd267-7aa5-41a4-ab06-d137ed425fa4_2' 
[2025-05-04T17:03:14.2244122Z][SCHEDULER][INFO]Scheduling job, id='HD_efddd267-7aa5-41a4-ab06-d137ed425fa4_1' 
[2025-05-04T17:03:14.3227458Z][SCHEDULER][INFO]Scheduling job, id='HD_efddd267-7aa5-41a4-ab06-d137ed425fa4_3' 
[2025-05

{'runId': 'HD_efddd267-7aa5-41a4-ab06-d137ed425fa4',
 'target': 'Udacity',
 'status': 'Completed',
 'startTimeUtc': '2025-05-04T17:03:11.581156Z',
 'endTimeUtc': '2025-05-04T17:21:22.047477Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'cb66464d-19c7-4f57-91f7-9332773f17d4',
  'user_agent': 'python/3.10.11 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_efddd267-7aa5-41a4-ab06-d137ed425fa4_7',
  'score': '0.9153920854576354',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_efddd267-7aa5-41a4-ab06-d137ed425fa4_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train'

In [13]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run.register_model(model_name = 'best_hd_model.pkl', model_path='outputs/model.joblib')

Run(Experiment: udacity-project,
Id: HD_efddd267-7aa5-41a4-ab06-d137ed425fa4_7,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-279736', subscription_id='d7f39349-a66b-446e-aba6-0053c2cf1c11', resource_group='aml-quickstarts-279736'), name=best_hd_model.pkl, id=best_hd_model.pkl:2, version=2, tags={}, properties={})

In [14]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '1', '--max_iter', '50']


In [15]:
print(best_run.get_metrics())

{'Max iterations:': 50, 'Regularization Strength:': 1.0, 'Accuracy': 0.9153920854576354}


In [16]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [29]:

from train_wv import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': 'AutoML_090de730-91bc-43a3-82f2-4a4df9b3310d_28'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': 'AutoML_090de730-91bc-43a3-82f2-4a4df9b3310d_28', 'run_id': 'AutoML_090de730-91bc-43a3-82f2-4a4df9b3310d_28'}


In [18]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)

In [19]:
# Submit your automl run

experiment = Experiment(ws, 'automl')
run = experiment.submit(config=automl_config, show_out=True)

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
2025-05-04 17:23:25.853026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746379407.249922    8958 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746379407.706313    8958 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746379411.477703    8958 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746379411.477769    8958 computation_placer.cc:177] computation placer already registered. Please check linkage

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_090de730-91bc-43a3-82f2-4a4df9b3310d,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2025/05/04 17:31:14 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [35]:
# Retrieve and save your best automl model.

#best_auto_run, model = run.get_output()
best_auto_run.register_model(model_name = 'automl_best_run', model_path='outputs/')
print(best_auto_run)

Run(Experiment: automl,
Id: AutoML_090de730-91bc-43a3-82f2-4a4df9b3310d_27,
Type: None,
Status: Completed)


In [27]:
print(best_auto_run.get_details())

{'runId': 'AutoML_090de730-91bc-43a3-82f2-4a4df9b3310d_27', 'target': 'local', 'status': 'Completed', 'startTimeUtc': '2025-05-04T17:54:16.34454Z', 'endTimeUtc': '2025-05-04T17:54:50.538489Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl\',\'compute_target\':\'local\',\'subscription_id\':\'d7f39349-a66b-446e-aba6-0053c2cf1c11\',\'region\':\'eastus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_090de730-91bc-43a3-82f2-4a4df9b3310d_27","experiment_name":null,"workspace_name":"quick-starts-ws-279736","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group_name

In [36]:
print(best_auto_run.get_metrics())

{'log_loss': 0.3317411450207488, 'average_precision_score_micro': 0.9814061083456702, 'norm_macro_recall': 0.49372661831800047, 'recall_score_micro': 0.9167830045523521, 'average_precision_score_macro': 0.8240135330007594, 'accuracy': 0.9167830045523521, 'average_precision_score_weighted': 0.9551617462169212, 'AUC_weighted': 0.9471450228296854, 'matthews_correlation': 0.5458665857013856, 'recall_score_weighted': 0.9167830045523521, 'f1_score_weighted': 0.9126303298731866, 'precision_score_macro': 0.8019316058599173, 'AUC_macro': 0.9471450228296854, 'precision_score_weighted': 0.9105401159388211, 'weighted_accuracy': 0.9589910377859248, 'f1_score_micro': 0.9167830045523521, 'AUC_micro': 0.9806721159802063, 'recall_score_macro': 0.7468633091590002, 'precision_score_micro': 0.9167830045523521, 'f1_score_macro': 0.7703353797123997, 'balanced_accuracy': 0.7468633091590002, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_090de730-91bc-43a3-82f2-4a4df9b3310d_27/accuracy_table', 

In [ ]:
cluster.delete()